In [1]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import datetime

from utils import normalize_adj, StandardScaler

# Model Architecture

In [2]:
# Create model
def gcn(signal_in, weights_hidden, weights_A, biases, hidden_num):
    
    signal_in = tf.transpose(signal_in, [1, 0, 2]) # node_num, ?batch, feature_in
    feature_len = signal_in.shape[2] # feature vector length at the node of the input graph
    
    i = 0
    while i < hidden_num:
        
        signal_in = tf.reshape(signal_in, [node_num, -1]) # node_num, batch*feature_in
        
        Adj = 0.5*(weights_A['A'+str(i)] + tf.transpose(weights_A['A'+str(i)]))
        Adj = normalize_adj(Adj)
        Z = tf.matmul(Adj, signal_in) # node_num, batch*feature_in 
        
        Z = tf.reshape(Z, [-1, int(feature_len)]) # node_num * batch, feature_in
        signal_output = tf.add(tf.matmul(Z, weights_hidden['h'+str(i)]), biases['b'+str(i)])
        signal_output = tf.nn.relu(signal_output) # node_num * batch, hidden_vec
        
        i += 1
        signal_in = signal_output # the sinal for next layer 
        feature_len = signal_in.shape[1] # feature vector length at hidden layers
    
    final_output = tf.add(tf.matmul(signal_output, weights_hidden['out']), biases['bout'])  # node_num * batch, horizon
    final_output = tf.reshape(final_output, [node_num, -1, horizon]) # node_num, batch, horizon
    final_output = tf.transpose(final_output, [1, 0, 2]) # batch, node_num, horizon
    final_output = tf.reshape(final_output, [-1, node_num*horizon]) # batch, node_num*horizon
 
    return final_output

In [3]:
def gcn_corr_final(feature_num_layer, horizon, learning_rate, decay, batch_size, keep, early_stop_th, training_epochs):
   
    feature_in = feature_num_layer[0] 
    
    early_stop_k = 0 # early stop patience
    display_step = 1 # frequency of printing results
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []

    tf.reset_default_graph()

    batch_size = batch_size
    early_stop_th = early_stop_th
    training_epochs = training_epochs

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, node_num, feature_in]) # X is the input signal
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # define dictionaries to store layers weight & bias
    i = 0
    weights_hidden = {}
    weights_A = {}
    biases = {}
    while i < len(feature_num_layer) - 1:
        weights_hidden['h'+str(i)] = tf.Variable(tf.random_normal([feature_num_layer[i], feature_num_layer[i + 1]]))
        biases['b'+str(i)] = tf.Variable(tf.random_normal([1, feature_num_layer[i + 1]]))
        weights_A['A'+str(i)] = tf.Variable(tf.random_normal([node_num, node_num]))
        i += 1
    
    weights_hidden['out'] = tf.Variable(tf.random_normal([feature_num_layer[-1], horizon]))
    biases['bout'] = tf.Variable(tf.random_normal([1, horizon]))
 
    # Construct model
    hidden_num = len(feature_num_layer) - 1
    pred= gcn(X, weights_hidden, weights_A, biases, hidden_num)
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)
    cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 

    pred_val= gcn(X, weights_hidden, weights_A, biases, hidden_num)
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 

    pred_tes= gcn(X, weights_hidden, weights_A, biases, hidden_num)
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
                                         
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                
                _, c = sess.run([optimizer, cost], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})

                avg_cost += c * batch_size #/ total_batch 
                
            # rest part of training dataset
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = np.sqrt(avg_cost / num_train)
            #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost))
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            c_val = np.sqrt(c_val[0])
            print("Validation RMSE: ", c_val)
            # testing
            c_tes, preds, Y_true = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            c_tes = np.sqrt(c_tes)

            if c_val < best_val:
                best_val = c_val
                # save model
                #saver.save(sess, './bikesharing_gcnn_ddgf')
                test_error = c_tes
                traing_error = avg_cost
                predic_res = preds
                early_stop_k = 0 # reset to 0

            # update early stopping patience
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)
        print("The test RMSE is ", test_error)
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return best_val, predic_res,Y_true,test_error

# Import Data

In [4]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

# Split Data into Training, Validation and Testing

In [5]:
node_num = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)

y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:node_num].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:node_num].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

n_input_vec = X_whole.shape[1] # e.g., 272 * 24
n_output_vec = Y_whole.shape[1] # each row represent a result


X_whole = np.reshape(X_whole, [X_whole.shape[0], node_num, feature_in])
num_samples = X_whole.shape[0]
num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]

X_test = X_whole[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole[num_train+num_val:num_train+num_val+num_test, :]



In [6]:
scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

# Hyperparameters

In [7]:
learning_rate = 0.005 # learning rate
decay = 0.9
batchsize = 100 # batch size 

feature_num_layer = [24, 10, 10, 5] # determine the number of hidden layers and the vector length at each node of each hidden layer

keep = 1 # drop out probability

early_stop_th = 20 # early stopping threshold, if validation RMSE not dropping in continuous 20 steps, break
training_epochs = 200 # total training epochs


# Training

In [8]:
a = datetime.datetime.now()

val_error, predic_res, test_Y, test_error = gcn_corr_final(feature_num_layer, horizon, learning_rate, decay, batchsize, keep, early_stop_th, training_epochs)


b = datetime.datetime.now()

print('Total training time: ', b-a)

#np.savetxt("prediction.csv", predic_res, delimiter = ',')
#np.savetxt("prediction_Y.csv", test_Y, delimiter = ',')


Epoch: 0001 Training RMSE= 48.557281030
Validation RMSE:  5.630722
Epoch: 0002 Training RMSE= 4.829149038
Validation RMSE:  4.81646
Epoch: 0003 Training RMSE= 4.005378480
Validation RMSE:  3.9481895
Epoch: 0004 Training RMSE= 3.678251639
Validation RMSE:  3.9025466
Epoch: 0005 Training RMSE= 3.601050654
Validation RMSE:  3.7120907
Epoch: 0006 Training RMSE= 3.546212844
Validation RMSE:  3.6575234
Epoch: 0007 Training RMSE= 3.480819403
Validation RMSE:  3.5907094
Epoch: 0008 Training RMSE= 3.392607091
Validation RMSE:  3.5773451
Epoch: 0009 Training RMSE= 3.323543121
Validation RMSE:  3.5701938
Epoch: 0010 Training RMSE= 3.274765211
Validation RMSE:  3.5582807
Epoch: 0011 Training RMSE= 3.242607740
Validation RMSE:  3.5466013
Epoch: 0012 Training RMSE= 3.215577654
Validation RMSE:  3.5389512
Epoch: 0013 Training RMSE= 3.192793006
Validation RMSE:  3.5263317
Epoch: 0014 Training RMSE= 3.172752166
Validation RMSE:  3.5018847
Epoch: 0015 Training RMSE= 3.155356977
Validation RMSE:  3.46513

Epoch: 0124 Training RMSE= 2.745292222
Validation RMSE:  3.0970426
Epoch: 0125 Training RMSE= 2.743428698
Validation RMSE:  3.122601
Epoch: 0126 Training RMSE= 2.742251469
Validation RMSE:  3.1342633
Epoch: 0127 Training RMSE= 2.742409456
Validation RMSE:  3.2147627
Epoch: 0128 Training RMSE= 2.740608287
Validation RMSE:  3.1226804
Epoch: 0129 Training RMSE= 2.740537530
Validation RMSE:  3.1276197
Epoch: 0130 Training RMSE= 2.739202962
Validation RMSE:  3.1025262
Epoch: 0131 Training RMSE= 2.738527566
Validation RMSE:  3.1168916
Epoch: 0132 Training RMSE= 2.736965254
Validation RMSE:  3.093938
Epoch: 0133 Training RMSE= 2.736964103
Validation RMSE:  3.152928
Epoch: 0134 Training RMSE= 2.735164161
Validation RMSE:  3.082569
Epoch: 0135 Training RMSE= 2.735844716
Validation RMSE:  3.1985042
Epoch: 0136 Training RMSE= 2.733857686
Validation RMSE:  3.1891146
Epoch: 0137 Training RMSE= 2.732535932
Validation RMSE:  3.1956096
Epoch: 0138 Training RMSE= 2.731338305
Validation RMSE:  3.1240535